# Spotify Data Collection API

In [6]:
# Import necessary libraries
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time

In [7]:
# Perform authentication (ADD YOUR CREDENTIALS HERE)
client_id = "cbf123007b33453cb4218e9a25e13f4c"
client_secret = "4398a3bfb2c044f595841eba30b63556"

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Utility Functions 

In [8]:
# Function to get the features of the track based on ID
def get_single_track_features(track_id, playlist_id):
    """
    Function to get the features and other information of a single track.
    """

    meta = sp.track(track_id)
    features = sp.audio_features(track_id)

    # Get the metadata of the track
    name = meta["name"]
    album = meta["album"]["name"]
    artist = meta["album"]["artists"][0]["name"]
    release_date = meta["album"]["release_date"]
    length = meta["duration_ms"]
    popularity = meta["popularity"]
    genres = meta["artists"]

    # Get the audio Features of the track
    acousticness = features[0]["acousticness"]
    key = features[0]["key"]
    danceability = features[0]["danceability"]
    energy = features[0]["energy"]
    instrumentalness = features[0]["instrumentalness"]
    liveness = features[0]["liveness"]
    loudness = features[0]["loudness"]
    speechiness = features[0]["speechiness"]
    tempo = features[0]["tempo"]
    time_signature = features[0]["time_signature"]
    mode = features[0]["mode"]
    valence = features[0]["valence"]
    # Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).

    return [
        track_id, 
        name,
        album,
        artist,
        release_date,
        length,
        popularity,
        key,
        acousticness,
        danceability,
        energy,
        instrumentalness,
        liveness,
        loudness,
        speechiness,
        tempo,
        valence,
        time_signature,
        mode,
        playlist_id,
    ]

In [9]:
# Function to create a dataframe of track information
def create_track_features_dataframe(tracks: list, playlist_id):
    """Function to generate a dataframe of song features from a list of songs

    Args:
        tracks (list): List of songs to make a dataframe of

    Returns:
        pandas.DataFrame: Dataframe of song features
    """

    # Iterate through the list of tracks
    all_track_data = []
    for track in tracks:
        time.sleep(0.5)
        track_data = get_single_track_features(track, playlist_id)
        all_track_data.append(track_data)

    return pd.DataFrame(
        all_track_data,
        columns=[
            "track_id", 
            "name",
            "album",
            "artist",
            "release_date",
            "length",
            "popularity",
            "key",
            "acousticness",
            "danceability",
            "energy",
            "instrumentalness",
            "liveness",
            "loudness",
            "speechiness",
            "tempo",
            "valence",
            "time_signature",
            "mode",
            "playlist_id",
        ],
    )

In [10]:
# Function to get the tracks from a playlist
def get_playlist_tracks(playlist_id):
    """Function to get the tracks from a playlist

    Args:
        playlist_id (str): ID of the playlist

    Returns:
        list: List of track IDs
    """

    playlist_tracks = sp.playlist(playlist_id, additional_types=("track",))

    track_ids = []
    for item in playlist_tracks["tracks"]["items"]:
        track = item["track"]
        track_ids.append(track["id"])

    return track_ids

### Perform Data Collection using playlists from Spotify. 

In [13]:
# Import the playlist data
playlist_df = pd.read_csv("../data/decade_wise_playlist.csv")
playlist_df.shape

(26, 5)

We are looking at 26 different playlists from various different decades. 

In [14]:
playlist_df.head()

,playlist_name,playlist_id,decade,user_id,language
0,60s Bollywood Hits,6mx7rVYF6ed2JTMegQ8SY0,1960,12163512469,hindi
1,60s Hindi Hit Songs,3dKv6gpADy34FI6rcP7DAT,1960,nq6nh207vo927cq4wpquchuor,hindi
2,70s Bollywood Hits,5p88YPX4eSead8hq0OCKAp,1970,12163512469,hindi
3,70s Evergreen Hindi,1arIwnl806bdxvrgTEuvLw,1970,juhere,hindi
4,All out 70s,37i9dQZF1DX9kVlnA5Si6s,1970,spotify,hindi


In [15]:
# Make a list of playlist IDs
playlist_id_list = playlist_df["playlist_id"].to_list()
playlist_id_list

['6mx7rVYF6ed2JTMegQ8SY0',
 '3dKv6gpADy34FI6rcP7DAT',
 '5p88YPX4eSead8hq0OCKAp',
 '1arIwnl806bdxvrgTEuvLw',
 '37i9dQZF1DX9kVlnA5Si6s',
 '37i9dQZF1DX5rOEFf3Iycd',
 '37i9dQZF1DXa1eCiO3E6Rr',
 '2o7YgEXIRPoQGHyFTuoJCS',
 '4Ja9naAu5wU8LC4xanc0pK',
 '37i9dQZF1DXa2huSXaKVkW',
 '70VQP8EHThS1uGVRq0qN9Z',
 '34nx68uXAiYvsrDdD1giJa',
 '1zpooNgRto3yMMDslyB1tn',
 '2poyZdIZgxPXymgKgH45Mu',
 'qi4jxs8hlj44noibkzc8p7epu',
 '37i9dQZF1DWZNJXX2UeBij',
 '3kRi9OHXNYJeSxYjqK2qfJ',
 '2Lw2dIL7pg5haDmtUSjzPq',
 '2Kj5NUtVetggUDHPIGC9U7',
 '4z6K8rFn6rjR5gddqWs0D2',
 '1pZmVlfPBUNMS19bBy3Qap',
 '4wJLkwU84uscxJ7SOlmUX1',
 '37i9dQZF1DX7BXvxNJkwzB',
 '37i9dQZF1DX0XUfTFmNBRM',
 '37i9dQZF1DX5q67ZpWyRrZ',
 '4PS7niVvIybq4KFGLDcy6D']

In [16]:
%%time 

# Create a dataframe with the first playlist ID \
print("Started the function for the first playlist..")
song_ids = get_playlist_tracks(playlist_id_list[0])
complete_playlist_data = create_track_features_dataframe(song_ids, playlist_id_list[0])


# Iterate through the playlist IDs 
for playlist_id in playlist_id_list[1:]: 
    print(f"Collecting data for playlist - {playlist_id}")
    
    try: 
        # Get the tracks for a particular playlist 
        song_ids = get_playlist_tracks(playlist_id)
        
        # Create a dataset for those tracks
        single_playlist_data = create_track_features_dataframe(song_ids, playlist_id)
        
        print(f"Collected {single_playlist_data.shape[0]} songs for this playlist")
        
        # Append to the existing data 
        complete_playlist_data = pd.concat([complete_playlist_data, single_playlist_data], axis=0)
            
    except Exception as e: 
        print("Encountered an error so skipping.")
        print(f"Error message: {e}")
        continue

print(f"Total songs collected - {complete_playlist_data.shape[0]}") 
        

Started the function for the first playlist..
Collected 100 songs for this playlist
Collected 57 songs for this playlist
Collected 45 songs for this playlist
Collected 100 songs for this playlist
Collected 100 songs for this playlist
Collected 100 songs for this playlist
Collected 90 songs for this playlist
Collected 42 songs for this playlist
Collected 100 songs for this playlist
Collected 22 songs for this playlist
Collected 100 songs for this playlist
Collected 42 songs for this playlist


HTTP Error for GET to https://api.spotify.com/v1/playlists/qi4jxs8hlj44noibkzc8p7epu with Params: {'fields': None, 'market': None, 'additional_types': 'track'} returned 404 due to Invalid playlist Id


Collected 100 songs for this playlist
Encountered an error so skipping.
Error message: http status: 404, code:-1 - https://api.spotify.com/v1/playlists/qi4jxs8hlj44noibkzc8p7epu?additional_types=track:
 Invalid playlist Id, reason: None
Collected 100 songs for this playlist
Collected 100 songs for this playlist
Collected 100 songs for this playlist
Collected 67 songs for this playlist
Collected 100 songs for this playlist
Collected 100 songs for this playlist
Collected 75 songs for this playlist
Collected 50 songs for this playlist
Collected 50 songs for this playlist
Collected 85 songs for this playlist
Collected 18 songs for this playlist
Total songs collected - 1903
Wall time: 25min 5s


In [17]:
complete_playlist_data.to_csv('complete_playlist_data.csv', index=False)

### Process the Complete songs data

In [20]:
song_data = pd.read_csv('../data/complete_playlist_data.csv')
song_data.shape

(1903, 20)

In [21]:
# Convert song data dates to datetime 
song_data

,track_id,name,album,artist,release_date,length,popularity,key,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,time_signature,mode,playlist_id
0,7eukWhfJ4WRkRnyeVyAFMs,Ehsaan Tera Hoga Mujh Par,Junglee (Original Motion Picture Soundtrack),Various Artists,1/28/2014,208986,12,4,0.985,0.380,0.389,0.000602,0.151,-7.241,0.0419,175.505,0.490,4,1,6mx7rVYF6ed2JTMegQ8SY0
1,3Jstl9LdDVzYyZwKouZ21W,Chahe Mujhe Koi Junglee Kahe,Junglee (Original Motion Picture Soundtrack),Various Artists,1/28/2014,213720,12,3,0.925,0.485,0.847,0.000000,0.110,-6.312,0.0736,147.565,0.803,4,1,6mx7rVYF6ed2JTMegQ8SY0
2,3MrRm2gBu7wUVPcZWKanai,Mera Saaya Saath Hoga,Mera Saaya (Original Motion Picture Soundtrack),Madan Mohan,12/1/1966,363200,12,2,0.982,0.557,0.422,0.100000,0.229,-10.230,0.0285,95.235,0.583,4,1,6mx7rVYF6ed2JTMegQ8SY0
3,7wJw4NRtBOXLhgmbxhLAs3,Jhoomka Gira Re,Mera Saaya (Original Motion Picture Soundtrack),Madan Mohan,12/1/1966,296825,13,8,0.901,0.598,0.787,0.000191,0.812,-4.920,0.1600,93.159,0.964,4,1,6mx7rVYF6ed2JTMegQ8SY0
4,1kKxvLPfgpG5NRJ9MAmYsu,Lag Ja Gale Se Phir,Woh Kaun Thi? (Original Motion Picture Soundtr...,Madan Mohan,11/1/2013,257613,37,8,0.796,0.312,0.452,0.000874,0.286,-11.872,0.0391,132.485,0.391,3,1,6mx7rVYF6ed2JTMegQ8SY0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1898,0sX0r664Hx5fEJcHNrkjyE,Pyar Do Pyar Lo - Janbaaz / Soundtrack Version,Janbaaz,Various Artists,1/1/1986,332293,34,11,0.357,0.460,0.874,0.000805,0.146,-8.898,0.2980,127.471,0.646,4,0,4PS7niVvIybq4KFGLDcy6D
1899,7mudoo9FXpSXeK4cxR32fJ,Dance Dance,Kasam Paida Karnewale Ki (Original Motion Pict...,Bappi Lahiri,11/23/1984,339304,21,7,0.173,0.604,0.637,0.000370,0.115,-12.944,0.1990,165.033,0.384,4,1,4PS7niVvIybq4KFGLDcy6D
1900,2QZs7FuTmfBCARyZ2w6UAD,Aap Ke Aa Jane Se,Khudgarz (Original Motion Picture Soundtrack),Rajesh Roshan,8/19/1987,432293,45,1,0.170,0.648,0.569,0.000089,0.159,-9.185,0.1500,123.614,0.678,4,0,4PS7niVvIybq4KFGLDcy6D
1901,4jqDpyUwVkcTJXufkuV79t,Yeh Mera Dil Yaar Ka Diwana,Don (Original Motion Picture Soundtrack),Kalyanji-Anandji,12/30/1977,253773,35,7,0.830,0.497,0.476,0.018400,0.332,-11.248,0.0624,141.460,0.521,4,0,4PS7niVvIybq4KFGLDcy6D


In [22]:
# Add year to the data
song_data['year'] = pd.to_datetime(song_data['release_date'], dayfirst=True, errors='coerce').dt.year

# Drop the null values 
song_data.dropna(inplace=True) 

# Add the decade to the data
song_data['decade'] = song_data['year'].apply(lambda x: x // 10 * 10)

# Convert length to seconds 
song_data['length'] = song_data['length'] / 1000


In [23]:
# Remove the decades 1950 and 1940 from the data 
song_data = song_data[(song_data['decade'] != 1940) & (song_data['decade'] != 1950)]
song_data.shape

(1887, 22)

In [27]:
# Remove the duplicate data 
song_data = song_data.drop_duplicates(subset='track_id')

In [29]:
song_data.shape

(1595, 22)

In [28]:
# Save the data with the new values 
song_data.to_csv('../data/complete_playlist_data.csv', index=False) 

In [30]:
# Create a dataframe with the decade wise data
decade_wise_data = song_data.groupby(['decade']).agg({'popularity': 'mean', 'danceability': 'mean', 'acousticness': 'mean', 'energy': 'mean', 'loudness': 'mean', 'speechiness': 'mean', 'valence': 'mean', 'tempo': 'mean', 'time_signature': 'mean', 'mode': 'mean'}).reset_index()
decade_wise_data

,decade,popularity,danceability,acousticness,energy,loudness,speechiness,valence,tempo,time_signature,mode
0,1960.0,24.263830,0.472366,0.890345,0.471379,-10.129060,0.069909,0.660549,110.948962,3.778723,0.706383
1,1970.0,20.788079,0.451934,0.804987,0.516815,-9.717166,0.110026,0.573988,113.019735,3.794702,0.655629
2,1980.0,35.640000,0.507029,0.636761,0.563727,-9.890977,0.067581,0.617696,112.181983,3.920000,0.634286
3,1990.0,42.448430,0.579108,0.476888,0.609386,-10.888695,0.060376,0.690887,118.277453,3.905830,0.511211
4,2000.0,45.231003,0.648818,0.303538,0.677611,-8.180149,0.071457,0.625036,114.597644,3.972644,0.613982
5,2010.0,35.044510,0.609680,0.398887,0.666148,-7.501742,0.071436,0.618220,117.815217,3.881306,0.602374
6,2020.0,57.524138,0.619593,0.488323,0.549883,-8.031241,0.064481,0.452657,114.529641,3.875862,0.703448
